# Data mapping with mistral-large model

In [35]:
%pip install -q llama-index-llms-mistralai llama-index-embeddings-mistralai

In [19]:
!pip install -q llama-index

In [56]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.core.multi_modal_llms.generic_utils import load_image_urls
from llama_index.core.llms import ChatMessage

## Define models

Here we are using Mistral large model hosted on La Plateforme, GPT4-V from OpenAI and the embedding model from Mistral too.

In [110]:
from google.colab import userdata


# Define LLM and embedding model
llm = MistralAI(api_key=userdata.get('MISTRAL_API_KEY'), model="mistral-large")
embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key=userdata.get('MISTRAL_API_KEY'))

# Define llama-index settings
Settings.llm = llm
Settings.embed_model = embed_model

In [111]:
tesseract_output_good_file="""REPUBLIQUE FRANCAISE

Liberté @ Egalité © Fraternité

Arrété municipal définitif n° 15.057

COMPTE TENU DE_;
La publication lo.

entant la circulation et le stationnement impasse Saint-Lucien, a
La notification le

compter du 9 mars 2015.

JPBIIS - 15.057 Le Maire de la Commune de La Courneuve,

Vu la loi n° 82-213 du 2 mars 1982 modifiée, relative aux droits et libertes des Communes,
des Départements et des Régions,

Vu le Code de la Route, notamment son article R.411.8, (R.417.10)

Vu le décret n° 86.475 du 14 mars 1986 relatif a I'exercice du pouvoir de police en matiére de
circulation routiére et modifiant certaines dispositions du Code de la Route,

Vu le décret n° 2010-578 du 31 mai 2010 modifiant le décret 2009-615 du 3 juin 2009 fixant la
liste des routes classées a grande circulation,

Vu le Code Général des Collectivités Territoriales, notamment les articles L.2213-1 et L.2213-
2,

Vu l'arrété et l’instruction interministériels sur la signalisation routiére, modifiés par les textes
subséquents,

Vu le réglement de voirie départementale adopté par I'Assemblée Départementale le 24
Septembre 1999,

Vu I'instruction interministérielle sur la signalisation routiére (livre | — signalisation des routes)
approuvée par arrété interministériel du 6 novembre 1992,

CONSIDERANT : que pour des raisons de sécurité, il y a lieu de réglementer la circulation et
le stationnement.

Sur proposition de Monsieur le Directeur Général de la Mairie de La Courneuve,
ARRETE

ARTICLE 1: Les conditions de circulation et de stationnement de l'impasse Saint-Lucien sont
modifiées a compter du 9 mars 2015.

ARTICLE 2: La vitesse des véhicules terrestres a moteur est limitée 4 20 km/h dans |'impasse
Saint-Lucien.

ARTICLE 3: Le mouvement de «tourne a gauche », depuis l'impasse Saint-Lucien sur
l'avenue Roger Salengro est interdit a tout véhicule.

ARTICLE 4: Le stationnement est interdit du cété des numéros impairs.
Le stationnement est également interdit aux véhicules affectés au transport de marchandise.

ARTICLE 5: La signalisation réglementaire est mise en place par le service Voirie et Réseaux

de Plaine Commune, Unité Territoriale de La Courneuve.
Hétel de ville

~—ARTICLE 6 : Toutes les infractions au présent arrété seront constatées par procés-verbaux et
avenue de la REBSERAMevenants poursuivis conformément a la loi.
93126 La Courneuve Cedex
tél. 01 49 92 60 00

toute correspondance doit :
atre adressée a M. le maire Liberté + Egalité + Fraternité

ARTICLE 7: Les agents de la voie publique sont chargés de l'application du présent arrété
dont la copie sera adressée a Monsieur le Commissaire de Police de la Courneuve.

et sera apposée sur les panneaux d'affichage administratif de la Ville de la Courneuve.

La Courneuve, le 19 février 2015

Vice-Président de Plaine Commune"""

In [112]:
tesseract_output_of_wrong_file_1="""SMe SPL AIG

REPUBLIQUE FRANCAISE
LIBERTE - EGALITE - FRATERNITE

VILLE DE VINCENNES

Direction générale VAL-DE-MARNE
des services techniques a .
Nok. 668 ARRETE REGLEMENTANT LE STATIONNEMENT
oo RUE SAULPIC

Le Maire de Vincennes, Conseiller régional d’fle-de-France,

Considérant la nécessité de déplacer I’ aire de livraison existante ;
Considérant qu’il convient de réglementer les opérations de manutention 4 Vincennes de maniére a limiter la géne
que ces opérations peuvent apporter a la circulation générale ;
Considérant qu’il y a lieu, compte tenu de la difficulté des conditions de circulation et de stationnement dans la
ville, de limiter la durée de ces opérations sur les zones arenas a cet effet du lundi au samedi de 7h a 20h;
A TE

ARTICLE I-— Le présent arrété abroge et remplace l’arrété n°3207 en date du 29 novembre 2005.
A compter du 15 ayril 2013 — du lundi au samedi _de7ha20h:

RUE SAULPIC cété impair sur les deux premiers emplacements de stationnement en partant de l’angle
de la voie et de ’avenue du Chateau :

Un stationnement sera réservé a l’arrét des véhicules effectuant des opérations de manutention.
Le stationnement des véhicules n’effectuant pas d’opérations de chargements ou déchargements de marchandise sera
déclaré comme génant, selon les termes de l'article R 417.10 du Code de la route et les véhicules en infraction pourront
faire l'objet d’un enlévement immédiat.
En dehors de ces jours et horaires, le stationnement des véhicules autres que ceux effectuant des manutentions
sera autorisé,

ARTICLE I - La Ville de VINCENNES procédera a la pose de la matérialisation de ces dispositions.

ARTICLE II - Les infractions au présent arrété seront constatées par des procés-verbaux.

ARTICLE IV - Le Directeur général des services de Ja ville, le Directeur général des services techniques de la
ville, le Commissaire de Police de Vincennes, sont chargés, chacun en ce qui le concerne, de |’application du
présent arrété,

ARTICLE V- Le présent arrété sera publié au Recueil des actes administratifs.

Vincennes, le ~9 AVR, 2013

wt Ny

Laurent LAFON,
Maire de Vincennes
Conseiller régional d’He-de-France

Hotel de Ville - 94304 VINCENNES Cedex — Standard : 01 43 98 65 00 — www.vincennes.fr"""

In [113]:
tesseract_output_of_wrong_file_2="""SERVICES Domaine : VOIRIE / TRAVAUX Département de
TECHNIQUES SEINE-ET-MARNE
ee REPUBLIQUE FRANCAISE Canton de
ADMINISTRATIF LIBERTE - EGALITE - FRATERNITE PONTAULT-COMBAULT

09,9,

Commune de

stuzisurrupaom | ARRETE PERMANENT DU MAIRE N°152/20| _Roissy.EN-BRIE

Objet : Réglementation de la circulation et du stationnement des véhicules
poids lourds de plus de 3,5t sur le quartier des Grands Champs

Le Maire de la Commune de Roissy-en-Brie

Considérant : la nécessité de réglementer la circulation et le stationnement des véhicules poids
lourds de plus de 3,5t, sur le quartier des Grands Champs.

ARRETE

Article 1: La circulation et le stationnement de tous véhicules de plus de 3,5t, sauf véhicules de
secours et de service, seront interdits sur le quartier des Grands Champs, a partir du
1° juillet 2020.

Article 2 : La mise en place et l’entretien de la signalisation réglementaire avec panneau type
B13 seront assurés par les services techniques de la ville de Roissy en Brie et placés
aux intersections suivanies : rue Aimé Bonpland, angle avenue J.B de Boismortier, rue
Michel Adamson, angle avenue J. B. de Boismortier, rue de Tournefort angle avenue J.
B. de Boismortier et rue de Jussieu angle avenue J. B. de Boismortier.

Article 3 : Tout contrevenant a ce présent arrété pourra faire l'objet de poursuites et étre
verbalisé. Les véhicules stationnant sur les lieux de la zone indiquée sur le présent
arrété pourront tre enlevés et déposés en fourriére (notamment, selon I'article R417-
10 du code de Ia route).

Article 4 : Mr le Maire de la ville de Roissy-en-Brie et le Commandant de Police sont chargés,
chacun en ce qui le concerne, de l'application du présent arrété qui n’entrera en
vigueur qu’aprés la mise en place de la signalisation réglementaire par les services
municipaux.

Article 5 :Mme - MM.- le Maire de Roissy-en-Brie,
le Directeur Départemental de la Sécurité Publique,
le Directeur du Centre d’Incendie et de Secours de Pontault-Combault,
le Commissaire Principal de la Police Nationale de Pontault-Combault,
le Chef de Service de la Police Municipale de Roissy-en-Brie,
sont chargés, chacun en ce qui le concerne de I'exécution du présent arrété.

Fait 4 Roissy-en-Brie, le 3 juin 2020

Pour le Maire,
Le premier adjoint délégué en charge du
& Aor pement urbain, des travaux, du
KS ean vie et gh ‘epvironnement.
az {
ed & fi th
sl 4 a
Gs 20) Jonathan/ZERUOUN
WE ET: me"""

In [114]:
## Create embedding out of the raw file

raw_text_embedding_good_file = embed_model.get_text_embedding(tesseract_output_good_file)
print(len(raw_text_embedding_good_file))

raw_text_embedding_wrong_file_1 = embed_model.get_text_embedding(tesseract_output_of_wrong_file_1)
raw_text_embedding_wrong_file_2 = embed_model.get_text_embedding(tesseract_output_of_wrong_file_2)

1024


In [115]:
## Create embedding of the clean xml template without values

template_xml_without_values = """<?xml version="1.0" encoding="UTF-8"?>

<d2:payload
  modelBaseVersion="3"
  lang="FR"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:d2="http://datex2.eu/schema/3/d2Payload"
  xmlns:com="http://datex2.eu/schema/3/common"
  xmlns:comx="http://datex2.eu/schema/3/commonExtension"
  xmlns:loc="http://datex2.eu/schema/3/locationReferencing"
  xmlns:locdx="https://raw.githubusercontent.com/MTES-MCT/dialog/main/docs/spec/datex2"
  xmlns="http://datex2.eu/schema/3/trafficRegulation"
  xsi:schemaLocation="http://datex2.eu/schema/3/d2Payload DATEXII_3_D2Payload.xsd"
  xsi:type="TrafficRegulationPublication"
  id="910928b3-58a1-4462-881b-53dec84602f9"
>
    <com:publicationTime></com:publicationTime>
    <com:publicationCreator>
        <com:country></com:country>
        <com:nationalIdentifier></com:nationalIdentifier>
    </com:publicationCreator>

    <trafficRegulationsFromCompetentAuthorities>
        <trafficRegulationOrder id="064ca240-2b58-73ea-8000-2d03faf5d44c" version="1">
    <description>
        <com:values>
            <com:value lang="fr"></com:value>
        </com:values>
    </description>

    <issuingAuthority>
        <com:values>
            <com:value lang="fr"></com:value>
        </com:values>
    </issuingAuthority>

    <regulationId></regulationId>
    <status></status>

    <trafficRegulation>
        <status></status>

        <typeOfRegulation xsi:type="AccessRestriction">
            <accessRestrictionType></accessRestrictionType>
        </typeOfRegulation>

        <condition xsi:type="ConditionSet">
            <operator></operator>

            <conditions xsi:type="ValidityCondition">
                <validityByOrder>
                    <com:validityStatus></com:validityStatus>
                    <com:validityTimeSpecification>
                        <com:overallStartTime></com:overallStartTime>
                        <com:overallEndTime></com:overallEndTime>
                    </com:validityTimeSpecification>
                </validityByOrder>
            </conditions>

            <conditions xsi:type="LocationCondition">
                <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                    <loc:linearWithinLinearElement>
                        <loc:linearElement xsi:type="loc:LinearElement">
                            <loc:roadName>
                                <com:values>
                                    <com:value lang="fr"></com:value>
                                </com:values>
                            </loc:roadName>
                        </loc:linearElement>

                        <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                            <loc:distanceAlong>0</loc:distanceAlong>
                            <loc:fromReferent>
                                <loc:referentIdentifier></loc:referentIdentifier>
                                <loc:referentName></loc:referentName>
                                <loc:referentType></loc:referentType>
                                <loc:pointCoordinates>
                                    <loc:latitude></loc:latitude>
                                    <loc:longitude></loc:longitude>
                                </loc:pointCoordinates>
                            </loc:fromReferent>
                        </loc:fromPoint>
                        <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                            <loc:distanceAlong></loc:distanceAlong>
                            <loc:fromReferent>
                                <loc:referentIdentifier></loc:referentIdentifier>
                                <loc:referentName></loc:referentName>
                                <loc:referentType></loc:referentType>
                                <loc:pointCoordinates>
                                    <loc:latitude></loc:latitude>
                                    <loc:longitude></loc:longitude>
                                </loc:pointCoordinates>
                            </loc:fromReferent>
                        </loc:toPoint>
                    </loc:linearWithinLinearElement>
                </locationByOrder>
            </conditions>
        </condition>
    </trafficRegulation>
</trafficRegulationOrder>

<trafficRegulationOrder id="random-id" version="1">
    <description>
        <com:values>
            <com:value lang="fr"></com:value>
        </com:values>
    </description>

    <issuingAuthority>
        <com:values>
            <com:value lang="fr"></com:value>
        </com:values>
    </issuingAuthority>

    <regulationId></regulationId>
    <status></status>

    <trafficRegulation>
        <status></status>

        <typeOfRegulation xsi:type="AccessRestriction">
            <accessRestrictionType></accessRestrictionType>
        </typeOfRegulation>

        <condition xsi:type="ConditionSet">
            <operator></operator>

            <conditions xsi:type="ConditionSet">
                <operator></operator>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus></com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime></com:overallStartTime>
                            <com:overallEndTime></com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus></com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime></com:overallStartTime>
                            <com:overallEndTime></com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus></com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime></com:overallStartTime>
                            <com:overallEndTime></com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus></com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime></com:overallStartTime>
                            <com:overallEndTime></com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus></com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime></com:overallStartTime>
                            <com:overallEndTime></com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus></com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime></com:overallStartTime>
                            <com:overallEndTime></com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
            </conditions>

            <conditions xsi:type="ConditionSet">
                <operator></operator>
                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr"></com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong></loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier></loc:referentIdentifier>
                                    <loc:referentName></loc:referentName>
                                    <loc:referentType></loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude></loc:latitude>
                                        <loc:longitude></loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong></loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier></loc:referentIdentifier>
                                    <loc:referentName></loc:referentName>
                                    <loc:referentType></loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude></loc:latitude>
                                        <loc:longitude></loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>

                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr"></com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong></loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier></loc:referentIdentifier>
                                    <loc:referentName></loc:referentName>
                                    <loc:referentType></loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude></loc:latitude>
                                        <loc:longitude></loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong></loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier></loc:referentIdentifier>
                                    <loc:referentName></loc:referentName>
                                    <loc:referentType></loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude></loc:latitude>
                                        <loc:longitude></loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>

                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr"></com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong></loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier></loc:referentIdentifier>
                                    <loc:referentName></loc:referentName>
                                    <loc:referentType></loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude></loc:latitude>
                                        <loc:longitude></loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong></loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier></loc:referentIdentifier>
                                    <loc:referentName></loc:referentName>
                                    <loc:referentType></loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude></loc:latitude>
                                        <loc:longitude></loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>

                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr"></com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:PercentageDistanceAlongLinearElement">
                                <loc:percentageDistanceAlong></loc:percentageDistanceAlong>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:PercentageDistanceAlongLinearElement">
                                <loc:percentageDistanceAlong></loc:percentageDistanceAlong>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>
            </conditions>

            <conditions xsi:type="DriverCondition">
                <negate></negate>
                <driverCharacteristicsType></driverCharacteristicsType>
            </conditions>
        </condition>
    </trafficRegulation>
</trafficRegulationOrder>

    </trafficRegulationsFromCompetentAuthorities>
</d2:payload>
"""

template_xml_embedding = embed_model.get_text_embedding(template_xml_without_values)
print(len(template_xml_embedding))

1024


In [116]:
system_prompt="""You are an expert in mapping municipal by-laws into xml files. YOU HAVE TO OUTPUT ONLY WITH THE FOLLWING TEMPLATE:
<?xml version="1.0" encoding="UTF-8"?>

<d2:payload
  modelBaseVersion="3"
  lang="FR"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:d2="http://datex2.eu/schema/3/d2Payload"
  xmlns:com="http://datex2.eu/schema/3/common"
  xmlns:comx="http://datex2.eu/schema/3/commonExtension"
  xmlns:loc="http://datex2.eu/schema/3/locationReferencing"
  xmlns:locdx="https://raw.githubusercontent.com/MTES-MCT/dialog/main/docs/spec/datex2"
  xmlns="http://datex2.eu/schema/3/trafficRegulation"
  xsi:schemaLocation="http://datex2.eu/schema/3/d2Payload DATEXII_3_D2Payload.xsd"
  xsi:type="TrafficRegulationPublication"
  id="910928b3-58a1-4462-881b-53dec84602f9"
>
    <com:publicationTime>2022-11-10T15:11:00.000Z</com:publicationTime>
    <com:publicationCreator>
        <com:country>fr</com:country>
        <com:nationalIdentifier>DiaLog</com:nationalIdentifier>
    </com:publicationCreator>

    <trafficRegulationsFromCompetentAuthorities>
        <trafficRegulationOrder id="064ca240-2b58-73ea-8000-2d03faf5d44c" version="1">
    <description>
        <com:values>
            <com:value lang="fr">ARRETE TEMPORAIRE ACT2023STD - 619-MB</com:value>
        </com:values>
    </description>

    <issuingAuthority>
        <com:values>
            <com:value lang="fr">Mairie de Saint-Denis</com:value>
        </com:values>
    </issuingAuthority>

    <regulationId>064ca240-2b58-73ea-8000-2d03faf5d44c</regulationId>
    <status>madeAndImplemented</status>

    <trafficRegulation>
        <status>active</status>

        <typeOfRegulation xsi:type="AccessRestriction">
            <accessRestrictionType>noEntry</accessRestrictionType>
        </typeOfRegulation>

        <condition xsi:type="ConditionSet">
            <operator>and</operator>

            <conditions xsi:type="ValidityCondition">
                <validityByOrder>
                    <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                    <com:validityTimeSpecification>
                        <com:overallStartTime>2023-08-11T18:00:00+02:00</com:overallStartTime>
                        <com:overallEndTime>2023-08-15T06:00:00+02:00</com:overallEndTime>
                    </com:validityTimeSpecification>
                </validityByOrder>
            </conditions>

            <conditions xsi:type="LocationCondition">
                <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                    <loc:linearWithinLinearElement>
                        <loc:linearElement xsi:type="loc:LinearElement">
                            <loc:roadName>
                                <com:values>
                                    <com:value lang="fr">Avenue du Stade de France, 93210 Saint-Denis</com:value>
                                </com:values>
                            </loc:roadName>
                        </loc:linearElement>

                        <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                            <loc:distanceAlong>0</loc:distanceAlong>
                            <loc:fromReferent>
                                <loc:referentIdentifier>start</loc:referentIdentifier>
                                <loc:referentName>Pont SNCF</loc:referentName>
                                <loc:referentType>landmark</loc:referentType>
                                <loc:pointCoordinates>
                                    <loc:latitude>48.917709</loc:latitude>
                                    <loc:longitude>2.361931</loc:longitude>
                                </loc:pointCoordinates>
                            </loc:fromReferent>
                        </loc:fromPoint>
                        <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                            <loc:distanceAlong>0</loc:distanceAlong>
                            <loc:fromReferent>
                                <loc:referentIdentifier>end</loc:referentIdentifier>
                                <loc:referentName>Rue Francis de Préssensé</loc:referentName>
                                <loc:referentType>intersection</loc:referentType>
                                <loc:pointCoordinates>
                                    <loc:latitude>48.919612</loc:latitude>
                                    <loc:longitude>2.361700</loc:longitude>
                                </loc:pointCoordinates>
                            </loc:fromReferent>
                        </loc:toPoint>
                    </loc:linearWithinLinearElement>
                </locationByOrder>
            </conditions>
        </condition>
    </trafficRegulation>
</trafficRegulationOrder>

<trafficRegulationOrder id="064ca247-fb24-7cf8-8000-0cf1af53270c" version="1">
    <description>
        <com:values>
            <com:value lang="fr">ARRETE TEMPORAIRE ACT2023STD - 365 SM</com:value>
        </com:values>
    </description>

    <issuingAuthority>
        <com:values>
            <com:value lang="fr">Mairie de Saint-Denis</com:value>
        </com:values>
    </issuingAuthority>

    <regulationId>064ca247-fb24-7cf8-8000-0cf1af53270c</regulationId>
    <status>madeAndImplemented</status>

    <trafficRegulation>
        <status>active</status>

        <typeOfRegulation xsi:type="AccessRestriction">
            <accessRestrictionType>noEntry</accessRestrictionType>
        </typeOfRegulation>

        <condition xsi:type="ConditionSet">
            <operator>and</operator>

            <conditions xsi:type="ConditionSet">
                <operator>or</operator>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime>2023-05-17T18:00:00+02:00</com:overallStartTime>
                            <com:overallEndTime>2023-05-19T06:00:00+02:00</com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime>2023-05-19T18:00:00+02:00</com:overallStartTime>
                            <com:overallEndTime>2023-05-23T06:00:00+02:00</com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime>2023-06-02T18:00:00+02:00</com:overallStartTime>
                            <com:overallEndTime>2023-06-05T06:00:00+02:00</com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime>2023-06-09T18:00:00+02:00</com:overallStartTime>
                            <com:overallEndTime>2023-06-12T06:00:00+02:00</com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime>2023-08-11T18:00:00+02:00</com:overallStartTime>
                            <com:overallEndTime>2023-08-15T06:00:00+02:00</com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
                <conditions xsi:type="ValidityCondition">
                    <validityByOrder>
                        <com:validityStatus>definedByValidityTimeSpec</com:validityStatus>
                        <com:validityTimeSpecification>
                            <com:overallStartTime>2023-11-03T18:00:00+02:00</com:overallStartTime>
                            <com:overallEndTime>2023-11-06T06:00:00+02:00</com:overallEndTime>
                        </com:validityTimeSpecification>
                    </validityByOrder>
                </conditions>
            </conditions>

            <conditions xsi:type="ConditionSet">
                <operator>or</operator>
                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr">Avenue des Fruitiers, 93210 Saint-Denis</com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong>0</loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier>start</loc:referentIdentifier>
                                    <loc:referentName>33</loc:referentName>
                                    <loc:referentType>referenceMarker</loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude>48.919245</loc:latitude>
                                        <loc:longitude>2.354415</loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong>0</loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier>end</loc:referentIdentifier>
                                    <loc:referentName>5</loc:referentName>
                                    <loc:referentType>referenceMarker</loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude>48.916503</loc:latitude>
                                        <loc:longitude>2.354752</loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>

                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr">Rue des Cheminots, 93210 Saint-Denis</com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong>0</loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier>start</loc:referentIdentifier>
                                    <loc:referentName>24</loc:referentName>
                                    <loc:referentType>referenceMarker</loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude>48.917786</loc:latitude>
                                        <loc:longitude>2.352652</loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong>0</loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier>end</loc:referentIdentifier>
                                    <loc:referentName>10</loc:referentName>
                                    <loc:referentType>referenceMarker</loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude>48.916110</loc:latitude>
                                        <loc:longitude>2.352746</loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>

                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr">Avenue François Mitterand, 93210 Saint-Denis</com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong>0</loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier>start</loc:referentIdentifier>
                                    <loc:referentName>17</loc:referentName>
                                    <loc:referentType>referenceMarker</loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude>48.918139</loc:latitude>
                                        <loc:longitude>2.352316</loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:DistanceFromLinearElementReferent">
                                <loc:distanceAlong>0</loc:distanceAlong>
                                <loc:fromReferent>
                                    <loc:referentIdentifier>end</loc:referentIdentifier>
                                    <loc:referentName>1</loc:referentName>
                                    <loc:referentType>referenceMarker</loc:referentType>
                                    <loc:pointCoordinates>
                                        <loc:latitude>48.918037</loc:latitude>
                                        <loc:longitude>2.356793</loc:longitude>
                                    </loc:pointCoordinates>
                                </loc:fromReferent>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>

                <conditions xsi:type="LocationCondition">
                    <locationByOrder xsi:type="loc:SingleRoadLinearLocation">
                        <loc:linearWithinLinearElement>
                            <loc:linearElement xsi:type="loc:LinearElement">
                                <loc:roadName>
                                    <com:values>
                                        <com:value lang="fr">Rue Federico Fellini, 93210 Saint-Denis</com:value>
                                    </com:values>
                                </loc:roadName>
                            </loc:linearElement>
                            <loc:fromPoint xsi:type="loc:PercentageDistanceAlongLinearElement">
                                <loc:percentageDistanceAlong>0</loc:percentageDistanceAlong>
                            </loc:fromPoint>
                            <loc:toPoint xsi:type="loc:PercentageDistanceAlongLinearElement">
                                <loc:percentageDistanceAlong>100</loc:percentageDistanceAlong>
                            </loc:toPoint>
                        </loc:linearWithinLinearElement>
                    </locationByOrder>
                </conditions>
            </conditions>

            <conditions xsi:type="DriverCondition">
                <negate>true</negate>
                <driverCharacteristicsType>localResident</driverCharacteristicsType>
            </conditions>
        </condition>
    </trafficRegulation>
</trafficRegulationOrder>

    </trafficRegulationsFromCompetentAuthorities>
</d2:payload>

use the values provide by the user"""

In [117]:
messages = [
    ChatMessage(role="system", content=system_prompt),
    ChatMessage(role="user", content=tesseract_output_good_file),
]
resp = MistralAI(api_key= userdata.get('MISTRAL_API_KEY'), max_tokens=50000, temperature=0.1).chat(messages)
print(resp)

assistant: <?xml version="1.0" encoding="UTF-8"?>

<d2:payload
  modelBaseVersion="3"
  lang="FR"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:d2="http://datex2.eu/schema/3/d2Payload"
  xmlns:com="http://datex2.eu/schema/3/common"
  xmlns:comx="http://datex2.eu/schema/3/commonExtension"
  xmlns:loc="http://datex2.eu/schema/3/locationReferencing"
  xmlns:locdx="https://raw.githubusercontent.com/MTES-MCT/dialog/main/docs/spec/datex2"
  xmlns="http://datex2.eu/schema/3/trafficRegulation"
  xsi:schemaLocation="http://datex2.eu/schema/3/d2Payload DATEXII_3_D2Payload.xsd"
  xsi:type="TrafficRegulationPublication"
  id="87654321-fghj-1234-5678-90abcd123456"
>
    <com:publicationTime>2023-03-10T15:11:00.000Z</com:publicationTime>
    <com:publicationCreator>
        <com:country>fr</com:country>
        <com:nationalIdentifier>Republique Francaise</com:nationalIdentifier>
    </com:publicationCreator>

    <trafficRegulationsFromCompetentAuthorities>
        <trafficRegulat

In [118]:
messages = [
    ChatMessage(role="system", content=system_prompt),
    ChatMessage(role="user", content=tesseract_output_of_wrong_file_1),
]
wrong_resp_1 = MistralAI(api_key= userdata.get('MISTRAL_API_KEY'), max_tokens=50000, temperature=0.1).chat(messages)
print(wrong_resp_1)

assistant: <?xml version="1.0" encoding="UTF-8"?>

<d2:payload
  modelBaseVersion="3"
  lang="FR"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:d2="http://datex2.eu/schema/3/d2Payload"
  xmlns:com="http://datex2.eu/schema/3/common"
  xmlns:comx="http://datex2.eu/schema/3/commonExtension"
  xmlns:loc="http://datex2.eu/schema/3/locationReferencing"
  xmlns:locdx="https://raw.githubusercontent.com/MTES-MCT/dialog/main/docs/spec/datex2"
  xmlns="http://datex2.eu/schema/3/trafficRegulation"
  xsi:schemaLocation="http://datex2.eu/schema/3/d2Payload DATEXII_3_D2Payload.xsd"
  xsi:type="TrafficRegulationPublication"
  id="3b1b1c2d-4e5f-6g7h-8i9j-5k0l1m2n3o"
>
    <com:publicationTime>2023-03-15T10:30:00.000Z</com:publicationTime>
    <com:publicationCreator>
        <com:country>FR</com:country>
        <com:nationalIdentifier>Ville de Vincennes</com:nationalIdentifier>
    </com:publicationCreator>

    <trafficRegulationsFromCompetentAuthorities>
        <trafficRegulationO

In [119]:
messages = [
    ChatMessage(role="system", content=system_prompt),
    ChatMessage(role="user", content=tesseract_output_of_wrong_file_2),
]
wrong_resp_2 = MistralAI(api_key= userdata.get('MISTRAL_API_KEY'), max_tokens=50000, temperature=0.1).chat(messages)
print(wrong_resp_2)

assistant: <?xml version="1.0" encoding="UTF-8"?>

<d2:payload
  modelBaseVersion="3"
  lang="FR"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:d2="http://datex2.eu/schema/3/d2Payload"
  xmlns:com="http://datex2.eu/schema/3/common"
  xmlns:comx="http://datex2.eu/schema/3/commonExtension"
  xmlns:loc="http://datex2.eu/schema/3/locationReferencing"
  xmlns:locdx="https://raw.githubusercontent.com/MTES-MCT/dialog/main/docs/spec/datex2"
  xmlns="http://datex2.eu/schema/3/trafficRegulation"
  xsi:schemaLocation="http://datex2.eu/schema/3/d2Payload DATEXII_3_D2Payload.xsd"
  xsi:type="TrafficRegulationPublication"
  id="87b12c1d-1234-5678-90ab-123e45678901"
>
    <com:publicationTime>2023-03-15T10:30:00.000Z</com:publicationTime>
    <com:publicationCreator>
        <com:country>fr</com:country>
        <com:nationalIdentifier>Roissy-en-Brie</com:nationalIdentifier>
    </com:publicationCreator>

    <trafficRegulationsFromCompetentAuthorities>
        <trafficRegulationOrd

In [120]:
## Create embedding out of the raw file

output_text_embedding = embed_model.get_text_embedding(resp.message.content)
print(len(output_text_embedding))

1024


Calculate cosine similarity between raw OCR output and the llm output

In [121]:
# raw_text_embedding
# template_xml_embedding
# output_text_embedding

# pip install numpy

import numpy as np
from numpy.linalg import norm

# Base (good) embedding
np_output_text_embedding = np.array(output_text_embedding)
np_template_xml_embedding = np.array(template_xml_embedding)
np_raw_text_embedding = np.array(raw_text_embedding_good_file)

output_vector = np_output_text_embedding-np_template_xml_embedding

cosine_similarity = np.dot(output_vector, np_raw_text_embedding)/(norm(output_vector)*norm(np_raw_text_embedding))

print("Base cosine similarity:", cosine_similarity)

# Wrong embedding 1
np_raw_text_embedding_1 = np.array(raw_text_embedding_wrong_file_1)

cosine_similarity = np.dot(output_vector, np_raw_text_embedding_1)/(norm(output_vector)*norm(np_raw_text_embedding_1))

print("False 1 cosine similarity:", cosine_similarity)

# Wrong embedding 2
np_raw_text_embedding_2 = np.array(raw_text_embedding_wrong_file_2)

cosine_similarity = np.dot(output_vector, np_raw_text_embedding_2)/(norm(output_vector)*norm(np_raw_text_embedding_2))

print("False 2 cosine similarity:", cosine_similarity)

Base cosine similarity: 0.2002946930616382
False 1 cosine similarity: 0.13084556818536006
False 2 cosine similarity: 0.12395714945421442
